In [1]:
import numpy as np
import torch
import os
from collections import OrderedDict
from torch.autograd import Variable
import torch.nn as nn
# from torch.nn import init
import functools
import torchvision.models as models

In [2]:
from torchvision import models
import torch
import torch.nn as nn
import torchvision
from PIL import Image
from torch.autograd import Variable
from torchvision import transforms, utils
import matplotlib.pyplot as plt

from skimage import io
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils


In [1]:
import os
import re

# Suppose you are trying to load pre-trained resnet model in directory- models\resnet

os.environ['TORCH_HOME'] = 'D:\dev\Pytorch_Models\models\\resnet' #setting the environment variable


In [ ]:
import json
import os
from collections import namedtuple
import zipfile

from PIL import Image


class CityscapeDataset(Dataset):
    """`Cityscapes <http://www.cityscapes-dataset.com/>`_ Dataset.

    Args:
        root (string): Root directory of dataset where directory ``leftImg8bit``
            
        split (string, optional): The image split to use, ``train``, ``train_extra`` or ``val``
       
        transform (callable, optional): A function/transform that takes in a PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``  
        transforms (callable, optional): A function/transform that takes input sample and its target as entry
            and returns a transformed version.

    Examples:

      
    """

    # Based on https://github.com/mcordts/cityscapesScripts
  

    def __init__(self, root, split='train_extra', transforms=None):
        super(Cityscapes, self).__init__(root, transforms, transform)
        self.images_dir = os.path.join(self.root, 'leftImg8bit', split)
        self.split = split
        self.images = []
        self.targets = []

        
        valid_modes = ("train", "train_extra", "val")

        
        for city in os.listdir(images_dir):
            img_dir = os.path.join(images_dir, city)
            images_city=[]
            for file_name in os.listdir(img_dir):
                
                images_city.append(os.path.join(img_dir, file_name))
            # re.split(\d+,input) splits by integer value
            images_city.sort( key= lambda text: int(re.split('(\d+)',text)[3]+re.split('(\d+)',text) [5] ))
            images.append(images_city)
                
[docs]    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
             image 
        """

        image = Image.open(self.images[index]).convert('RGB')
        if self.transforms is not None:
            image = self.transforms(image)

        return image


    def __len__(self):
        return len(self.images)



In [ ]:
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

In [14]:
composed_transforms=transforms.Compose([transforms.Resize((224, 224)),                              transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),                                        transforms.ToTensor()])

In [ ]:
dataloader = DataLoader(transformed_dataset, batch_size=4,
                        shuffle=True, num_workers=4)